In [1]:
import pandas as pd
import fiona
import geopandas as gpd
from sqlalchemy import create_engine
import numpy as np
#import dedupe
from dotenv import load_dotenv
import os
import pyArango
import usaddress
from usaddress import tag
from scourgify import normalize_address_record
import re
import json

In [2]:
from pyArango.connection import *
conn = Connection(username="root", password="0505")

In [3]:
load_dotenv()
PG_CONNECT = os.getenv("PG_CONNECT")

In [4]:
# create arangoDB database, or open it if it already exists
try:
    corp_db = conn.createDatabase(name="corp_data")
    # from command line:
    # arangoimport --file '''path to CorpData.csv''' --collection corp_data --create-collection true --type csv --server.database corp_data
    # arangoimport --file '''path to CorpIndividualExport.csv''' --collection corp_individual --create-collection true --type csv --server.database corp_data

except:
    corp_db = conn["corp_data"]

In [ ]:
# arangoDB database structure:

# corp_db: arangoDB database
# corp_db["corp_data"]: arangoDB collection
# corp_db["corp_data"][17777999]: arangoDB document (one piece of data, or one corporation)

In [5]:
# extracting the values of the corp_data_processed collection
val_aql = "FOR x IN corp_data_processed RETURN x"
value_query_result = corp_db.AQLQuery(val_aql,rawResults=True, batchSize = 1000)
col_value = {}
ind_val = 0

for value in value_query_result:
    col_value[ind_val] = value
    ind_val += 1
    
# create dataframe from dictionary of dictionaries
corp = pd.DataFrame.from_dict(data = col_value, orient = 'index')

In [13]:
# extracting the values of the indiv_data_processed collection
indiv_aql = "FOR x IN indiv_data_processed RETURN x"
indiv_query_result = corp_db.AQLQuery(indiv_aql,rawResults=True, batchSize = 1000)
col_indiv = {}
ind_indiv = 0

for indiv in indiv_query_result:
    col_indiv[ind_indiv] = indiv
    ind_indiv += 1

indiv = pd.DataFrame.from_dict(data = col_indiv, orient = 'index')

In [6]:
#process geodatabase
l3_assess_geo = gpd.read_file("../MassGIS_L3_Parcels_gdb/MassGIS_L3_Parcels.gdb", driver='FileGDB', layer=2) #L3_ASSESS
l3_assess = pd.DataFrame(l3_assess_geo)
print(l3_assess)

            PROP_ID            LOC_ID  BLDG_VAL  LAND_VAL  OTHER_VAL  \
0            D9 4 3  F_335507_2850670    111000     98900          0   
1            D9 5 3  F_335433_2850987    140200     90500       6600   
2           D9 6 14  F_335682_2851486    344900    184100       9200   
3           E10 1 5  F_337304_2851580    375800    200900      18500   
4          E10 2 23  F_336769_2852989         0     29500          0   
...             ...               ...       ...       ...        ...   
2536875  0104966006  F_781897_2959209    551200         0          0   
2536876  0104972008  F_781998_2959152    536400         0          0   
2536877  1904214002  F_756936_2929234    437900         0          0   
2536878  1200335006  F_769944_2942985    436500         0          0   
2536879  0903674006  F_766578_2944741    383700         0          0   

         TOTAL_VAL    FY  LOT_SIZE   LS_DATE  LS_PRICE  ...          STYLE  \
0           209900  2021      0.39  19930802  106000.0  .

In [ ]:
#*****EXPERIMENTAL, DON'T RUN*****

# Parcel data from MassGIS Data
# https://www.mass.gov/forms/massgis-request-statewide-parcel-data

# Residential Land Use Codes from MA Dept of Revenue
# https://www.mass.gov/files/documents/2016/08/wr/classificationcodebook.pdf
# Codes are 101*-109*, 031*, and 013*
# Often include suffixes (letters, zeroes or no character), thus regex *?

USE_CODES = '^1[0-1][1-9]*?|^013*?|^031*?'
# BOS_CODES = '^R[1-4]$|^RC$|^RL$|^CD$|^A$'

# medparse = lambda x: pd.datetime.strptime(x, '%Y%m%d')

def read_res(file_dict):
    df = pd.DataFrame()
    for town, file, in file_dict.items():
        town_df = gpd.read_file(file).drop('geometry', axis='columns')
        town_df['town'] = town
        df = df.append(town_df, ignore_index=True)
    return df